In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns
from IPython.display import Image
import pickle
import caffe
import sys
import os

import random
import numpy as np
from numpy import (array, dot, arccos, clip)
from numpy.linalg import norm

import cv2

/work/sagarj/Work/caffe-fr-chairs/python/caffe/pycaffe.py:13: RuntimeWarning: to-Python converter for boost::shared_ptr<caffe::Net<float> > already registered; second conversion method ignored.
  from ._caffe import Net, SGDSolver, NesterovSolver, AdaGradSolver, \
/work/sagarj/Work/caffe-fr-chairs/python/caffe/pycaffe.py:13: RuntimeWarning: to-Python converter for boost::shared_ptr<caffe::Blob<float> > already registered; second conversion method ignored.
  from ._caffe import Net, SGDSolver, NesterovSolver, AdaGradSolver, \
/work/sagarj/Work/caffe-fr-chairs/python/caffe/pycaffe.py:13: RuntimeWarning: to-Python converter for boost::shared_ptr<caffe::Solver<float> > already registered; second conversion method ignored.
  from ._caffe import Net, SGDSolver, NesterovSolver, AdaGradSolver, \


In [2]:
with open("../Data/categoryIndex_places205.csv" , 'rb') as f:
    cats = f.readlines()
catDict = {}
for c in cats:
    comps = c.strip().split(',')
    idx = comps[0].split(' ')[1]
    lbl = comps[0].split(' ')[0]
    catDict[idx] = {}
    catDict[idx]['label'] = lbl
    catDict[idx]['tnomy'] = comps[-1]


In [3]:
catDict[catDict.keys()[1]]

{'label': '/p/palace', 'tnomy': 'L'}

In [4]:
with open("../Data/cityDf.pkl" , 'rb') as f:
    cityImages = pickle.load(f)

In [5]:
len(cityImages.keys())

3164

In [6]:
cityImages[cityImages.keys()[1]]

{'key': '513d7b59fdc9f03587006b2f',
 'label': 0,
 'path': '/work/sagarj/Work/BellLabs/streetview/PPImages/513d7b59fdc9f03587006b2f.jpg',
 'trueSkill': [18.990548645401752]}

In [7]:
YesLabels = ['/b/bridge', '/f/forest_road', '/v/viaduct', '/h/highway', '/f/forest_path', '/s/sky', '/t/tree_farm', '/r/rainforest', '/r/runway', '/r/railroad_track', '/m/mountain_snowy', '/w/wind_farm', '/b/bamboo_forest', '/o/orchard', '/r/racecourse', '/v/valley', '/f/field/wild', '/c/corn_field', '/t/track/outdoor', '/s/snowfield']
NoLabels = ['/c/courthouse', '/f/formal_garden', '/h/hotel/outdoor', '/g/gas_station', '/b/building_facade', '/p/playground', '/a/alley', '/o/office_building', '/c/construction_site', '/r/residential_neighborhood', '/p/patio', '/d/driveway', '/m/motel', '/a/apartment_building/outdoor', '/h/hospital', '/p/plaza', '/m/mansion', '/y/yard', '/i/inn/outdoor', '/c/courtyard']


In [8]:
# If you get "No module named _caffe", either you have not built pycaffe or you have the wrong path.

model_root = "/datasets_1/sagarj/BellLabs/caffe_models/places/"

imagenet_mean = model_root + 'places205CNN_mean.binaryproto'

meanFile = "/datasets_1/sagarj/BellLabs/Data/places205CNN_mean.npy"

logfile = "../Data/PlacesFeatExtractStreetview.txt"


#Size of images
IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
NCHANNELS = 3
meanR = 105.487823486
meanG = 113.741088867
meanB = 116.060394287

meanMat = np.zeros((NCHANNELS,IMAGE_WIDTH,IMAGE_HEIGHT))
meanMat[0,:,:].fill(meanR)
meanMat[1,:,:].fill(meanG)
meanMat[2,:,:].fill(meanB)
#227

def transform_img(img, img_width=IMAGE_WIDTH, img_height=IMAGE_HEIGHT):

    #Histogram Equalization
    img[:, :, 0] = cv2.equalizeHist(img[:, :, 0])
    img[:, :, 1] = cv2.equalizeHist(img[:, :, 1])
    img[:, :, 2] = cv2.equalizeHist(img[:, :, 2])

    #Image Resizing
    img = cv2.resize(img, (img_width, img_height), interpolation = cv2.INTER_CUBIC)

    return img


def predictImage(imgPath , net, transformer):
    

    path = imgPath.strip()
    im = caffe.io.load_image(path)
    net.blobs['data'].data[...] = transformer.preprocess('data', im)
    net.forward()
    #out1 = net.blobs['prob'].data
    out2 = net.blobs['fc7'].data
    #print(out2.shape)
    #out = np.concatenate((out1,out2.reshape(1,-1)),axis =1)
    out = out2
    #print(out.shape)
    return out


def getPlaces(Data ):
    caffe.set_mode_gpu()    
    model_def = model_root + 'places205CNN_deploy_upgraded.prototxt'#'test.prototxt'
    model_weights = model_root +'places205CNN_iter_300000_upgraded.caffemodel'#'caffe_sentibank_train_iter_250000'
    #model_def = model_root + 'places205VGG16/deploy_10.prototxt'
    #model_weights = model_root +'places205VGG16/snapshot_iter_765280.caffemodel'
    net = caffe.Net(model_def,      # defines the structure of the model
                model_weights,  # contains the trained weights
                caffe.TEST)     # use test mode (e.g., don't perform dropout)

    net.blobs['data'].reshape(1,3,IMAGE_WIDTH,IMAGE_HEIGHT)
    
    transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape}) 
    transformer.set_transpose('data', (2,0,1))
    transformer.set_channel_swap('data', (2,1,0))
    transformer.set_raw_scale('data', 255.0)

    for k in Data:
        imgPath = Data[k]['path']
        #print imgPath
        path = imgPath.strip()
        if not os.path.exists(path):
            featArray = np.zeros((1,205))
        else:
            im = transformer.preprocess('data',caffe.io.load_image(path))
            net.blobs['data'].data[...] = im - meanMat
            net.forward()
            #featArray = net.blobs['fc7'].data.copy()
            featArray = np.squeeze(net.blobs['prob'].data.copy())
        Data[k]['Places'] = featArray
    return Data

In [9]:
cityImages = getPlaces(cityImages)

/usr/local/lib/python2.7/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [26]:
#cityImages[cityImages.keys()[1]]

In [22]:
np.argsort(cityImages[cityImages.keys()[1]]['Places'])[-5:]

array([ 67, 135,  60,  79,  92])

In [38]:
count = 0
for k in cityImages:
    top5idx = np.argsort(cityImages[k]['Places'])[-5:]
    labels = [catDict[str(i)]['label'] for i in top5idx]
    yays = set(labels).intersection(YesLabels)
    nays = set(labels).intersection(NoLabels)
    
    if (len(yays) - len(nays)) > 0:
        cityImages[k]['AugFlag'] = True
        count +=1 
    else:
        cityImages[k]['AugFlag'] = False
print count

767


# Create LMDB for images that are positive for augmentatble labels

In [50]:
import os
import glob
import random
import numpy as np
import pickle

import cv2
import sys
caffe_root = '/work/sagarj/caffe-rc5/'  # this file should be run from {caffe_root}/examples (otherwise change this line)
sys.path.insert(0, caffe_root + 'python')

import caffe
from caffe.proto import caffe_pb2
import lmdb

#Size of images
IMAGE_WIDTH = 227
IMAGE_HEIGHT = 227

def transform_img(img, img_width=IMAGE_WIDTH, img_height=IMAGE_HEIGHT):

    #Histogram Equalization
    img[:, :, 0] = cv2.equalizeHist(img[:, :, 0])
    img[:, :, 1] = cv2.equalizeHist(img[:, :, 1])
    img[:, :, 2] = cv2.equalizeHist(img[:, :, 2])

    #Image Resizing
    img = cv2.resize(img, (img_width, img_height), interpolation = cv2.INTER_CUBIC)

    return img


def make_datum(img, label):
    #image is numpy.ndarray format. BGR instead of RGB
    return caffe_pb2.Datum(
        channels=3,
        width=IMAGE_WIDTH,
        height=IMAGE_HEIGHT,
        label=label,
        data=np.rollaxis(img, 2).tostring())

def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)


In [46]:
testKeys = random.sample(list(cityImages.keys()), int(0.2*len(cityImages.keys())))

In [47]:
trainKeys = [k for k in cityImages.keys() if k not in testKeys]

In [48]:
len(testKeys) , len(trainKeys)

(632, 2532)

In [44]:
train_lmdb = '../Data/train_lmdb_beauty_validatedAugment'
validation_lmdb = '../Data/validation_lmdb_beauty_validatedAugment'
log = "augValide.log"

In [45]:
AugDir = "/work/sagarj/Work/BellLabs/streetview/USAEasternAugImages/"

In [55]:
import random
train_data = {}
for k in trainKeys:
    src = [cityImages[k]['path']]
    if cityImages[k]['AugFlag']:
        if os.path.exists(AugDir + k):
            images = os.listdir(AugDir + k)
            random.shuffle(images)
            for i in images:
                src.append(AugDir + k + "/" + i)
    train_data[k] = src
        
test_data = {}
for k in testKeys:
    src = [cityImages[k]['path']]
    if cityImages[k]['AugFlag']:
        if os.path.exists(AugDir + k):
            images = os.listdir(AugDir + k)
            random.shuffle(images)
            for i in images:
                src.append(AugDir + k + "/" + i)
    test_data[k] = src
    

In [60]:
#train_data[train_data.keys()[101]]
#cityImages[train_data.keys()[101]]

In [61]:
# print ('Creating train_lmdb')
# f = open(log,'w')
# in_db = lmdb.open(train_lmdb, map_size=int(1e12))
# with in_db.begin(write=True) as in_txn:
#     for in_idx, (k, img_path) in enumerate(train_data.items()):
#         label = cityImages[k]['label']
#         for p in img_path:
#             img = cv2.imread(p, cv2.IMREAD_COLOR)
#             img = transform_img(img, img_width=IMAGE_WIDTH, img_height=IMAGE_HEIGHT)
#             datum = make_datum(img, label)
#             in_txn.put('{:0>5d}'.format(in_idx), datum.SerializeToString())
#             line = '{:0>5d}'.format(in_idx) + ':' + p + "\n"
#             f.write(line)
# in_db.close()


# print ('\nCreating validation_lmdb')

# in_db = lmdb.open(validation_lmdb, map_size=int(1e12))
# with in_db.begin(write=True) as in_txn:
#     for in_idx, (k, img_path) in enumerate(test_data.items()):
#         label = cityImages[k]['label']
#         for p in img_path:
#             img = cv2.imread(p, cv2.IMREAD_COLOR)
#             img = transform_img(img, img_width=IMAGE_WIDTH, img_height=IMAGE_HEIGHT)
#             datum = make_datum(img, label)
#             in_txn.put('{:0>5d}'.format(in_idx), datum.SerializeToString())
#             line = '{:0>5d}'.format(in_idx) + ':' + p + "\n"
#             f.write(line)
# in_db.close()
# f.close()
# print ('\nFinished processing all images')


Creating train_lmdb

Creating validation_lmdb

Finished processing all images
